# Knowledge Distillation for LLMs - Single GPU Lab

This notebook demonstrates **Knowledge Distillation** using NVIDIA TensorRT Model Optimizer on a single GPU.

## What is Knowledge Distillation?

Knowledge Distillation is a model compression technique where:
- A smaller **student model** learns to mimic a larger **teacher model**
- The student learns from both:
  1. Ground truth labels (standard supervised learning)
  2. Soft predictions from the teacher (knowledge transfer)

### In this lab:
- **Teacher**: Llama-3.2-3B-Instruct (3.2B parameters)
- **Student**: Llama-3.2-1B (1.2B parameters)
- **Goal**: Compress the teacher's knowledge into a 2.6× smaller model

---

## 📋 Lab Setup

### Prerequisites
- Single GPU with ~40GB VRAM (e.g., A100, A6000)
- Python 3.10+
- CUDA 11.8+

### Learning Objectives
1. Understand knowledge distillation process
2. Implement dataset preprocessing for LLM training
3. Configure and run distributed training
4. Evaluate distilled model performance

---

## 🔧 Step 1: GPU Configuration

**IMPORTANT**: Configure which GPU to use BEFORE importing PyTorch!

Check available GPUs with `nvidia-smi` or `nvitop` and select a free GPU.

In [1]:
# For faster library installation
!pip install uv nvitop
# Install TensorRT Model Optimizer with HuggingFace support
!uv pip install -U nvidia-modelopt[hf]

!uv pip uninstall numpy transformers
# Install additional dependencies
!uv pip install pyarrow 'transformers<5.0' 'trl>=0.23.0' 'numpy<2.0' bitsandbytes accelerate

Defaulting to user installation because normal site-packages is not writeable
Resolved 90 packages in 904ms                                        
⠙ Preparing packages... (0/5)                                                   
⠙ Preparing packages... (0/5)-------------------     0 B/858.14 MiB          
⠙ Preparing packages... (0/5)------------------- 14.88 KiB/858.14 MiB        
⠙ Preparing packages... (0/5)------------------- 30.88 KiB/858.14 MiB        
⠙ Preparing packages... (0/5)------------------- 46.88 KiB/858.14 MiB        
⠙ Preparing packages... (0/5)------------------- 62.88 KiB/858.14 MiB        
⠙ Preparing packages... (0/5)------------------- 78.88 KiB/858.14 MiB        
⠙ Preparing packages... (0/5)------------------- 94.88 KiB/858.14 MiB        
⠙ Preparing packages... (0/5)------------------- 110.88 KiB/858.14 MiB       
⠙ Preparing packages... (0/5)------------------- 126.88 KiB/858.14 MiB       
⠙ Preparing packages... (0/5)------------------- 142.88 KiB/858.14 Mi

In [2]:
import os
from dataclasses import dataclass

GPU_ID = 4  # Change this to your available GPU (0-7)

os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU_ID)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print(f"🎯 GPU Configuration:")
print(f"   Selected GPU: {GPU_ID}")
print(f"   CUDA_VISIBLE_DEVICES = {os.environ['CUDA_VISIBLE_DEVICES']}")
print(f"\n⚠️  After this setting, GPU {GPU_ID} will appear as 'cuda:0' in PyTorch")
print(f"   This is normal and expected!\n")

🎯 GPU Configuration:
   Selected GPU: 4
   CUDA_VISIBLE_DEVICES = 4

⚠️  After this setting, GPU 4 will appear as 'cuda:0' in PyTorch
   This is normal and expected!



## 📦 Step 2: Import Libraries

Now that GPU is configured, we can import PyTorch and other libraries.

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets

# TensorRT Model Optimizer imports
import modelopt.torch.distill as mtd
import modelopt.torch.opt as mto
from modelopt.torch.distill.plugins.huggingface import KDTrainer, LMLogitsLoss

print("✅ Libraries imported successfully!")
print(f"==> PyTorch version: {torch.__version__}")
print(f"==> Transformers version: {transformers.__version__}")
print(f"==> CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"==> Visible GPU count: {torch.cuda.device_count()}")
    print(f"==> Device 0 name: {torch.cuda.get_device_name(0)}")
    print(f"==> Total memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"\n✅ SUCCESS: Using GPU {GPU_ID} as 'cuda:0'")
else:
    print("❌ ERROR: CUDA not available!")

/mnt/nfs/kpaksaran/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


✅ Libraries imported successfully!
==> PyTorch version: 2.9.1+cu128
==> Transformers version: 4.57.1
==> CUDA available: True
==> Visible GPU count: 1
==> Device 0 name: NVIDIA A100-SXM4-40GB
==> Total memory: 39.39 GB

✅ SUCCESS: Using GPU 4 as 'cuda:0'


/mnt/nfs/kpaksaran/.venv/lib/python3.11/site-packages/modelopt/torch/__init__.py:36: UserWarning: transformers version 4.57.1 is incompatible with nvidia-modelopt and may cause issues. Please install recommended version with `pip install nvidia-modelopt[hf]` if working with HF models.
  _warnings.warn(


## ⚙️ Step 3: Training Configuration

Configure the model paths and training hyperparameters.

In [3]:
@dataclass
class ModelArguments:
    """Model Configuration"""
    # Teacher: Larger model we distill FROM
    teacher_name_or_path: str = "meta-llama/Llama-3.2-3B-Instruct"
    
    # Student: Smaller model we distill TO
    student_name_or_path: str = "meta-llama/Llama-3.2-1B"


# Create model configuration
model_args = ModelArguments()

# Configure training arguments
training_args = transformers.TrainingArguments(
    output_dir="./llama3.2-1b-distilled-1gpu",
    do_train=True,
    do_eval=True,
    
    # Training duration
    max_steps=3200,  # ~2 epochs with our dataset
    
    # Checkpointing
    save_strategy="steps",
    save_steps=500,
    
    # Batch size - adjust based on GPU memory
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,  # Effective batch = 2 * 16 = 32
    
    # Optimizer settings
    optim="adamw_torch",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    
    # Mixed precision (faster training, less memory)
    bf16=True,
    tf32=False,
    
    # Logging and evaluation
    logging_steps=50,
    eval_steps=400,
    report_to="none",
    
    # Data processing
    dataloader_drop_last=True,
)

print("✅ Configuration:")
print("="*80)
print(f"  Teacher: {model_args.teacher_name_or_path}")
print(f"  Student: {model_args.student_name_or_path}")
print(f"\n📊 Training Parameters:")
print(f"  Batch size per device:      {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation:      {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size:       {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate:              {training_args.learning_rate}")
print(f"  Max steps:                  {training_args.max_steps}")
print(f"\n💾 Output: {training_args.output_dir}")
print("="*80)

✅ Configuration:
  Teacher: meta-llama/Llama-3.2-3B-Instruct
  Student: meta-llama/Llama-3.2-1B

📊 Training Parameters:
  Batch size per device:      2
  Gradient accumulation:      16
  Effective batch size:       32
  Learning rate:              2e-05
  Max steps:                  3200

💾 Output: ./llama3.2-1b-distilled-1gpu


/mnt/nfs/kpaksaran/.venv/lib/python3.11/site-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


## 📊 Step 4: Load and Prepare Dataset

We'll use the **smol-smoltalk-Interaction-SFT** dataset, which contains conversational query-answer pairs.

### Dataset Preprocessing
The raw dataset has columns: `query`, `answer`, `source`

We need to:
1. Format into chat template (user/assistant messages)
2. Tokenize the text
3. Create `input_ids`, `attention_mask`, and `labels`

In [4]:
print("Loading dataset...")

# Load the dataset from HuggingFace
dset = datasets.load_dataset("ReactiveAI/smol-smoltalk-Interaction-SFT", split="train")

# Split into training and evaluation sets
dset_splits = dset.train_test_split(train_size=12800, test_size=1280, seed=420)
dset_train, dset_eval = dset_splits["train"], dset_splits["test"]

print(f"✅ Dataset loaded!")
print(f"  Training samples: {len(dset_train):,}")
print(f"  Evaluation samples: {len(dset_eval):,}")
print(f"\n📝 Sample data:")
print(f"  Query: {dset_train[0]['query'][:100]}...")
print(f"  Answer: {dset_train[0]['answer'][:100]}...")

Loading dataset...
✅ Dataset loaded!
  Training samples: 12,800
  Evaluation samples: 1,280

📝 Sample data:
  Query: What are Data visualization types....
  Answer: Data visualization types are diverse and can be categorized based on their purpose, structure, and f...


## 🔤 Step 5: Load Tokenizer and Preprocess Dataset

The tokenizer converts text into tokens that the model can understand.

In [5]:
print("Loading tokenizer...")

tokenizer = AutoTokenizer.from_pretrained(model_args.teacher_name_or_path, use_fast=True)

# Configure padding
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded")
print(f"  Vocab size: {len(tokenizer):,}")
print(f"  Pad token: '{tokenizer.pad_token}'")
print(f"  EOS token: '{tokenizer.eos_token}'")

Loading tokenizer...
✅ Tokenizer loaded
  Vocab size: 128,256
  Pad token: '<|eot_id|>'
  EOS token: '<|eot_id|>'


### Define Preprocessing Function

This function:
1. Converts query/answer into chat format
2. Applies the chat template
3. Tokenizes with truncation
4. Creates labels for language modeling

In [6]:
def format_sample(sample):
    """
    Format and tokenize a dataset sample.
    
    Args:
        sample: Dict with 'query' and 'answer' keys
        
    Returns:
        Dict with 'input_ids', 'attention_mask', and 'labels'
    """
    # Create chat messages
    messages = [
        {"role": "user", "content": sample["query"]},
        {"role": "assistant", "content": sample["answer"]},
    ]
    
    # Apply chat template
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    
    # Tokenize with truncation
    tokenized = tokenizer(
        text, 
        truncation=True, 
        max_length=512,  # Limit sequence length
        padding=False    # Dynamic padding in data collator
    )
    
    # Create labels (copy of input_ids for language modeling)
    tokenized["labels"] = list(tokenized["input_ids"])
    
    return tokenized

print("✅ Preprocessing function defined")

# Test the function on one sample
test_sample = format_sample(dset_train[0])
print(f"\n📊 Test output:")
print(f"  Input IDs length: {len(test_sample['input_ids'])}")
print(f"  Attention mask length: {len(test_sample['attention_mask'])}")
print(f"  Labels length: {len(test_sample['labels'])}")

✅ Preprocessing function defined

📊 Test output:
  Input IDs length: 486
  Attention mask length: 486
  Labels length: 486


### Apply Preprocessing to Dataset

This will tokenize all samples in the dataset.

In [7]:
print("Tokenizing datasets...")
print("This may take a few minutes...\n")

# Apply to training set
dset_train = dset_train.map(
    format_sample, 
    remove_columns=dset_train.column_names,  # Remove original columns
    num_proc=4,  # Parallel processing
    desc="Tokenizing train set"
)

# Apply to evaluation set
dset_eval = dset_eval.map(
    format_sample,
    remove_columns=dset_eval.column_names,
    num_proc=4,
    desc="Tokenizing eval set"
)

print(f"\n✅ Tokenization complete!")
print(f"  Train set columns: {dset_train.column_names}")
print(f"  Train set features: {dset_train.features}")

Tokenizing datasets...
This may take a few minutes...



Tokenizing eval set (num_proc=4): 100%|██████████| 1280/1280 [00:00<00:00, 1480.15 examples/s]


✅ Tokenization complete!
  Train set columns: ['input_ids', 'attention_mask', 'labels']
  Train set features: {'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}


## 🤖 Step 6: Load Student Model

Load the smaller student model that will learn from the teacher.

In [8]:
print(f"Loading student model: {model_args.student_name_or_path}")
print("This may take a few minutes...\n")

student_model = AutoModelForCausalLM.from_pretrained(
    model_args.student_name_or_path,
    torch_dtype=torch.bfloat16 if training_args.bf16 else torch.float32,
    device_map={"":0}  # Load on first GPU
)

student_params = sum(p.numel() for p in student_model.parameters())

print(f"✅ Student model loaded!")
print(f"  Parameters: {student_params:,} ({student_params/1e9:.2f}B)")
print(f"  Device: {next(student_model.parameters()).device}")
print(f"  Memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

Loading student model: meta-llama/Llama-3.2-1B
This may take a few minutes...



`torch_dtype` is deprecated! Use `dtype` instead!


✅ Student model loaded!
  Parameters: 1,235,814,400 (1.24B)
  Device: cuda:0
  Memory: 2.30 GB


## 👨‍🏫 Step 7: Load Teacher Model & Configure Distillation

Load the larger teacher model and set up knowledge distillation.

In [9]:
print(f"Loading teacher model: {model_args.teacher_name_or_path}")
print("This may take a few minutes...\n")

teacher_model = AutoModelForCausalLM.from_pretrained(
    model_args.teacher_name_or_path,
    torch_dtype=torch.bfloat16 if training_args.bf16 else torch.float32,
    device_map={"":0}  # Load on first GPU
)

teacher_params = sum(p.numel() for p in teacher_model.parameters())

print(f"✅ Teacher model loaded!")
print(f"  Parameters: {teacher_params:,} ({teacher_params/1e9:.2f}B)")
print(f"  Device: {next(teacher_model.parameters()).device}")
print(f"  Compression ratio: {teacher_params/student_params:.2f}x")
print(f"  Total GPU memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

Loading teacher model: meta-llama/Llama-3.2-3B-Instruct
This may take a few minutes...



Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.20it/s]


✅ Teacher model loaded!
  Parameters: 3,212,749,824 (3.21B)
  Device: cuda:0
  Compression ratio: 2.60x
  Total GPU memory: 8.29 GB


### Configure Knowledge Distillation

Set up the distillation loss function and convert the student model.

In [10]:
print("Configuring knowledge distillation...\n")

# Configure KD loss
kd_config = {
    "teacher_model": teacher_model,
    "criterion": LMLogitsLoss(),  # KL-divergence on logits
}

# Enable ModelOpt checkpointing for HuggingFace
mto.enable_huggingface_checkpointing()

# Convert student to distillation model
model = mtd.convert(student_model, mode=[("kd_loss", kd_config)])

# Fix generation config warnings
model.generation_config.temperature = None
model.generation_config.top_p = None

print("✅ Distillation configured!")
print(f"  Loss function: LMLogitsLoss (KL-divergence)")
print(f"  Student will learn from:")
print(f"    1. Ground truth labels (standard loss)")
print(f"    2. Teacher's soft predictions (distillation loss)")

Configuring knowledge distillation...

ModelOpt save/restore enabled for `transformers` library.
ModelOpt save/restore enabled for `diffusers` library.
ModelOpt save/restore enabled for `peft` library.
✅ Distillation configured!
  Loss function: LMLogitsLoss (KL-divergence)
  Student will learn from:
    1. Ground truth labels (standard loss)
    2. Teacher's soft predictions (distillation loss)


## 🚀 Step 8: Setup Trainer and Start Training

Create the trainer with the appropriate data collator.

In [11]:
print("Setting up trainer...\n")

# Use DataCollatorForSeq2Seq which handles labels properly
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

# Create KD Trainer
trainer = KDTrainer(
    model,
    training_args,
    train_dataset=dset_train,
    eval_dataset=dset_eval,
    data_collator=data_collator,
    processing_class=tokenizer,
)

print("✅ Trainer ready!")
print(f"\n📊 Training schedule:")
print(f"  Total samples: {len(dset_train):,}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Total steps: {training_args.max_steps:,}")
print(f"  Estimated time: ~{training_args.max_steps * 1.5 / 60:.0f} minutes")

Setting up trainer...

ModelOpt save/restore enabled for `transformers` library.
ModelOpt save/restore enabled for `diffusers` library.
ModelOpt save/restore enabled for `peft` library.


The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Trainer ready!

📊 Training schedule:
  Total samples: 12,800
  Batch size: 2
  Gradient accumulation: 16
  Effective batch size: 32
  Total steps: 3,200
  Estimated time: ~80 minutes


### Start Training

This will train for 3,200 steps (~2 epochs).

**Note**: Training will take approximately 50-60 minutes on a single A100 GPU.

In [12]:
print("="*80)
print("STARTING TRAINING")
print("="*80)
print("\n⏱️  This will take approximately 50-60 minutes...\n")

# Start training
trainer.train()

print("\n" + "="*80)
print("✅ TRAINING COMPLETE!")
print("="*80)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


STARTING TRAINING

⏱️  This will take approximately 50-60 minutes...



Step,Training Loss
50,32.598700
100,15.759200
150,7.144900
200,6.212300
250,5.882700
300,5.660100
350,5.558500
400,5.384700
450,4.393300
500,4.414600


Memory usage at training step 1, device=0: memory (MB) | allocated:  1.32e+04 | max_allocated:  1.79e+04 | reserved:  1.81e+04 | max_reserved:  1.81e+04
Saved ModelOpt state to ./llama3.2-1b-distilled-1gpu/checkpoint-500/modelopt_state.pth
Saved ModelOpt state to ./llama3.2-1b-distilled-1gpu/checkpoint-1000/modelopt_state.pth
Saved ModelOpt state to ./llama3.2-1b-distilled-1gpu/checkpoint-1500/modelopt_state.pth
Saved ModelOpt state to ./llama3.2-1b-distilled-1gpu/checkpoint-2000/modelopt_state.pth
Saved ModelOpt state to ./llama3.2-1b-distilled-1gpu/checkpoint-2500/modelopt_state.pth
Saved ModelOpt state to ./llama3.2-1b-distilled-1gpu/checkpoint-3000/modelopt_state.pth
Saved ModelOpt state to ./llama3.2-1b-distilled-1gpu/checkpoint-3200/modelopt_state.pth

✅ TRAINING COMPLETE!


## 💾 Step 9: Save the Distilled Model

In [21]:
print("Saving model...\n")

# Save training state
trainer.save_state()

# Save the distilled student model
# Note: export_student parameter may not work with all ModelOpt versions
# If it fails, the model is already saved in checkpoints
try:
    # Try to export student model
    if hasattr(model, 'export'):
        exported_model = model.export()
        trainer.model = exported_model
        trainer.save_model(training_args.output_dir)
    else:
        # Fallback: save without export
        # The model checkpoints already contain the distilled weights
        print("⚠️  Model doesn't have export() method")
        print("✅ Using checkpoint-3200 as final model (already saved during training)")
        
        # Copy the latest checkpoint to output directory
        import shutil
        checkpoint_dir = f"{training_args.output_dir}/checkpoint-3200"
        if os.path.exists(checkpoint_dir):
            # The checkpoint is already there, just note it
            print(f"✅ Final model available at: {checkpoint_dir}")
except Exception as e:
    print(f"⚠️  Export failed: {e}")
    print("✅ Model weights saved in checkpoints during training")

print(f"\n✅ Model saved successfully!")
print(f"\n📂 Available checkpoints:")
for item in sorted(os.listdir(training_args.output_dir)):
    item_path = os.path.join(training_args.output_dir, item)
    if os.path.isdir(item_path) and item.startswith('checkpoint'):
        print(f"  - {item}")
        
print(f"\n💡 To use the model, load from the latest checkpoint:")
print(f"   AutoModelForCausalLM.from_pretrained('{training_args.output_dir}/checkpoint-3200')")

Saving model...

⚠️  Model doesn't have export() method
✅ Using checkpoint-3200 as final model (already saved during training)
✅ Final model available at: ./llama3.2-1b-distilled-1gpu/checkpoint-3200

✅ Model saved successfully!

📂 Available checkpoints:
  - checkpoint-1000
  - checkpoint-1500
  - checkpoint-2000
  - checkpoint-2500
  - checkpoint-3000
  - checkpoint-3200
  - checkpoint-500

💡 To use the model, load from the latest checkpoint:
   AutoModelForCausalLM.from_pretrained('./llama3.2-1b-distilled-1gpu/checkpoint-3200')


## 📈 Step 10: Comprehensive Evaluation - Before & After Distillation

We'll evaluate three models to demonstrate the effectiveness of knowledge distillation:
1. **Teacher Model** (3B params) - Our target/reference
2. **Baseline Student** (1B params) - Before distillation (untrained)
3. **Distilled Student** (1B params) - After distillation (trained)

This comparison will show:
- How much the baseline student lags behind the teacher
- How much knowledge distillation helps close the gap
- The effectiveness of our training process

In [22]:
import math
import pandas as pd

print("="*80)
print("COMPREHENSIVE MODEL EVALUATION")
print("="*80)
print()

# Helper function to evaluate a model
def evaluate_model(model, model_name):
    """Evaluate a model and return metrics"""
    print(f"Evaluating {model_name}...")
    
    eval_trainer = transformers.Trainer(
        model,
        transformers.TrainingArguments(
            output_dir=training_args.output_dir,
            per_device_eval_batch_size=2,
            bf16=True,
        ),
        eval_dataset=dset_eval,
        data_collator=data_collator,
        processing_class=tokenizer,
    )
    
    results = eval_trainer.evaluate()
    perplexity = math.exp(results['eval_loss'])
    
    return {
        'loss': results['eval_loss'],
        'perplexity': perplexity,
        'runtime': results['eval_runtime'],
        'samples_per_sec': results['eval_samples_per_second']
    }

# Store results
results_dict = {}

# 1. Evaluate Teacher Model
print("\n" + "="*80)
print("1️⃣  TEACHER MODEL (3B - Target Performance)")
print("="*80)
results_dict['Teacher (3B)'] = evaluate_model(teacher_model, "Teacher")
print(f"✅ Teacher Loss: {results_dict['Teacher (3B)']['loss']:.4f}")
print(f"✅ Teacher Perplexity: {results_dict['Teacher (3B)']['perplexity']:.2f}")

# 2. Evaluate Baseline Student (before distillation)
print("\n" + "="*80)
print("2️⃣  BASELINE STUDENT (1B - Before Distillation)")
print("="*80)
print("Loading baseline student model (untrained)...\n")

baseline_student = AutoModelForCausalLM.from_pretrained(
    model_args.student_name_or_path,
    torch_dtype=torch.bfloat16,
    device_map={"":0}
)

results_dict['Baseline Student (1B)'] = evaluate_model(baseline_student, "Baseline Student")
print(f"✅ Baseline Loss: {results_dict['Baseline Student (1B)']['loss']:.4f}")
print(f"✅ Baseline Perplexity: {results_dict['Baseline Student (1B)']['perplexity']:.2f}")

# Free memory
del baseline_student
torch.cuda.empty_cache()

# 3. Evaluate Distilled Student (after training)
print("\n" + "="*80)
print("3️⃣  DISTILLED STUDENT (1B - After Distillation)")
print("="*80)
print("Loading distilled model...\n")

# Load from the latest checkpoint
distilled_model_path = f"{training_args.output_dir}/checkpoint-{training_args.max_steps}"
if not os.path.exists(distilled_model_path):
    # Fallback to the output directory itself
    distilled_model_path = training_args.output_dir
    
print(f"Loading from: {distilled_model_path}")

distilled_model = AutoModelForCausalLM.from_pretrained(
    distilled_model_path,
    torch_dtype=torch.bfloat16,
    device_map={"":0}
)

results_dict['Distilled Student (1B)'] = evaluate_model(distilled_model, "Distilled Student")
print(f"✅ Distilled Loss: {results_dict['Distilled Student (1B)']['loss']:.4f}")
print(f"✅ Distilled Perplexity: {results_dict['Distilled Student (1B)']['perplexity']:.2f}")

# Calculate improvements
print("\n" + "="*80)
print("📊 COMPARISON SUMMARY")
print("="*80)

# Create comparison table
df = pd.DataFrame(results_dict).T
df.index.name = 'Model'
df = df[['loss', 'perplexity', 'samples_per_sec']]
df.columns = ['Eval Loss', 'Perplexity', 'Samples/sec']

print("\n" + df.to_string())

# Calculate improvements
baseline_ppl = results_dict['Baseline Student (1B)']['perplexity']
distilled_ppl = results_dict['Distilled Student (1B)']['perplexity']
teacher_ppl = results_dict['Teacher (3B)']['perplexity']

improvement = ((baseline_ppl - distilled_ppl) / baseline_ppl) * 100
gap_closed = ((baseline_ppl - teacher_ppl) - (distilled_ppl - teacher_ppl)) / (baseline_ppl - teacher_ppl) * 100

print("\n" + "="*80)
print("🎯 KEY INSIGHTS")
print("="*80)
print(f"\n1. Performance Improvement:")
print(f"   Distillation improved perplexity by {improvement:.1f}%")
print(f"   (from {baseline_ppl:.2f} to {distilled_ppl:.2f})")

print(f"\n2. Gap to Teacher:")
print(f"   Baseline gap: {baseline_ppl - teacher_ppl:.2f} perplexity points")
print(f"   Distilled gap: {distilled_ppl - teacher_ppl:.2f} perplexity points")
print(f"   Knowledge distillation closed {gap_closed:.1f}% of the gap!")

print(f"\n3. Efficiency:")
print(f"   Distilled model has {student_params/1e9:.2f}B parameters")
print(f"   Teacher has {teacher_params/1e9:.2f}B parameters")
print(f"   Achieved {(1 - (distilled_ppl - teacher_ppl)/(baseline_ppl - teacher_ppl)) * 100:.1f}% of teacher quality")
print(f"   with only {(student_params/teacher_params)*100:.1f}% of parameters!")

print("\n" + "="*80)
print("✅ Evaluation Complete!")
print("="*80)

The model is already on multiple devices. Skipping the move to device specified in `args`.


COMPREHENSIVE MODEL EVALUATION


1️⃣  TEACHER MODEL (3B - Target Performance)
Evaluating Teacher...


/mnt/nfs/kpaksaran/.venv/lib/python3.11/site-packages/modelopt/torch/distill/distillation_model.py:312: UserWarning: Teacher's Module `LlamaForCausalLM` already has an intermediate output stored. This is expected when `DistillationModel.compute_kd_loss` is not called in eval mode.
  warnings.warn(


✅ Teacher Loss: 1.6820
✅ Teacher Perplexity: 5.38

2️⃣  BASELINE STUDENT (1B - Before Distillation)
Loading baseline student model (untrained)...

Evaluating Baseline Student...


The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Baseline Loss: 2.3376
✅ Baseline Perplexity: 10.36

3️⃣  DISTILLED STUDENT (1B - After Distillation)
Loading distilled model...

Loading from: ./llama3.2-1b-distilled-1gpu/checkpoint-3200
Restored ModelOpt state from ./llama3.2-1b-distilled-1gpu/checkpoint-3200/modelopt_state.pth


The model is already on multiple devices. Skipping the move to device specified in `args`.


Evaluating Distilled Student...


✅ Distilled Loss: 1.8450
✅ Distilled Perplexity: 6.33

📊 COMPARISON SUMMARY

                        Eval Loss  Perplexity  Samples/sec
Model                                                     
Teacher (3B)             1.682030    5.376461       21.560
Baseline Student (1B)    2.337583   10.356179       42.929
Distilled Student (1B)   1.844975    6.327943       43.541

🎯 KEY INSIGHTS

1. Performance Improvement:
   Distillation improved perplexity by 38.9%
   (from 10.36 to 6.33)

2. Gap to Teacher:
   Baseline gap: 4.98 perplexity points
   Distilled gap: 0.95 perplexity points
   Knowledge distillation closed 80.9% of the gap!

3. Efficiency:
   Distilled model has 1.24B parameters
   Teacher has 3.21B parameters
   Achieved 80.9% of teacher quality
   with only 38.5% of parameters!

✅ Evaluation Complete!


## 🧪 Step 11: Test Inference

Test the distilled model with sample prompts.

In [23]:
print("Testing distilled model inference...\n")

# Test prompt
test_messages = [
    {"role": "user", "content": "What is knowledge distillation and why is it useful?"}
]

test_prompt = tokenizer.apply_chat_template(
    test_messages, 
    tokenize=False, 
    add_generation_prompt=True
)

print("📝 Prompt:")
print(test_prompt)
print("\n" + "="*80)

# Tokenize input
inputs = tokenizer(test_prompt, return_tensors="pt").to(distilled_model.device)

# Generate response
print("Generating response...\n")
with torch.no_grad():
    outputs = distilled_model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# Decode response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("🤖 Response:")
print("="*80)
print(response)
print("="*80)
print("\n✅ Inference complete!")

Testing distilled model inference...

📝 Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Nov 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is knowledge distillation and why is it useful?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Generating response...

🤖 Response:
system

Cutting Knowledge Date: December 2023
Today Date: 13 Nov 2025

user

What is knowledge distillation and why is it useful?assistant

Knowledge distillation is the process of selectively extracting the most valuable and relevant information from a vast amount of knowledge sources, such as books, articles, research papers, and expert opinions, and distilling it into a concise and actionable form. The goal of knowledge distillation is to identify the most important insights, patterns, and principles that can be applied to a specific problem or issue.

The benefits of knowledge distillation include:

1. **Reducing info

## 🎉 Lab Complete!

### Summary

You have successfully:
1. ✅ Configured a single GPU environment
2. ✅ Loaded and preprocessed a conversational dataset
3. ✅ Set up knowledge distillation between teacher and student models
4. ✅ Trained a distilled 1B parameter model from a 3B teacher
5. ✅ Evaluated the distilled model's performance
6. ✅ Tested inference with the distilled model

### Key Takeaways

- **Model Compression**: Reduced from 3.2B to 1.2B parameters (2.6× smaller)
- **Knowledge Transfer**: Student learned from both ground truth and teacher predictions
- **Practical Skills**: Dataset preprocessing, distributed training setup, model evaluation

### Next Steps

1. **Compare Performance**: Benchmark the distilled model against the teacher
2. **Optimize Further**: Try different hyperparameters or longer training
3. **Deploy**: Use the smaller model for efficient inference
4. **Experiment**: Try different teacher-student pairs or datasets

---

**Happy distilling! 🚀**